In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split ,cross_val_score
#import gdown
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.datasets import make_regression
import numpy as np 
SEED = 1412 

In [ ]:
My_data = make_regression(n_samples=10000,n_features=5,noise=20,shuffle=False,random_state=SEED)

In [ ]:
X,y = My_data

In [ ]:
print(type(X),type(y))
print(np.shape(X),np.shape(y))

In [ ]:
My_df = pd.DataFrame(X)
My_df['Answer'] = y 
My_df.head()

In [ ]:
corr = My_df.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(10, 10))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap='RdBu', annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns)
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

LR = LinearRegression()
SDGR = SGDRegressor()
RFR = RandomForestRegressor()

split 1

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.modified import HybridBatAlgorithm


def get_hyperparameters(x):
    """
    Function to get hyperparameters from x value.
    base_estimator, n_estimators, learning_rate, and loss
    """
    print(f'x -> \n{x}')
    
    bases = (LR, SDGR, RFR)
    base_estimator = bases[int(x[0] * (len(bases)-1))]
    n_estimators = int(x[1] * 100 + 1)
    learning_rate = x[2] * 3 + 0.000001
    losses = ('linear', 'square', 'exponential')
    loss = losses[int(x[3] * 2)]


    params = {
        'base_estimator': base_estimator,
        'n_estimators': n_estimators,
        'learning_rate' : learning_rate,
        'loss': loss,
    }
    print(f'Para -> \n{params}')

    return params


def get_regressor(x):
    """     generate regressor berdasarkan hyperparameter yang dipilih.     """
    params = get_hyperparameters(x)
    return AdaBoostRegressor(**params)

In [ ]:
class ABRHyperparameterOptimization(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=4, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=5, n_jobs=-1)
        return scores.mean()

In [14]:
problem = ABRHyperparameterOptimization(X_train, y_train)

# akan menjalankan maksimalisasi untuk 100 iterasi pada permasalahan dengan populasi 10
task = Task(problem, max_iters=100, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = HybridBatAlgorithm(population_size=10, seed=1234)
best_params_HBA, best_accuracy = algorithm.run(task)
 
print('Best parameters:', get_hyperparameters(best_params_HBA))

In [ ]:
default_model = AdaBoostRegressor()
best_model_HBA = get_regressor(best_params_HBA)

default_model.fit(X_train, y_train)
best_model_HBA.fit(X_train, y_train)

In [ ]:
default_score_train = default_model.score(X_train, y_train)
best_score_HBA_train = best_model_HBA.score(X_train, y_train)

print('Default model accuracy:', default_score_train)
print('Best model accuracy:', best_score_HBA_train)

In [ ]:
default_score_test = default_model.score(X_test, y_test)
best_score_HBA_test = best_model_HBA.score(X_test, y_test)

print('Default model accuracy:', default_score_test)
print('Best model accuracy:', best_score_HBA_test)